Read the data and create shapefiles with extra attributes

In [1]:
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point
from geopandas import GeoSeries, GeoDataFrame

filepath="C:\\zsofia\\Amsterdam\\GitHub\\eEcoLiDAR\\myPhD_annotation\\Data\\"

header=pd.read_csv(filepath+'Phragmites_australis_header.csv',sep='\t')
species=pd.read_csv(filepath+'Phragmites_australis_species.csv',sep='\t')

- Convert species info into one line information

In [2]:
SpeciesGroups=species.groupby("PlotObservationID")["Species name"].apply(' '.join).reset_index()
LayerGroups=species.groupby("PlotObservationID")["Layer"].apply(lambda x: ', '.join(x.astype(str))).reset_index()
CoverGroups=species.groupby("PlotObservationID")["Cover %"].apply(lambda x: ', '.join(x.astype(str))).reset_index()
CoverCodeGroups=species.groupby("PlotObservationID")["Cover code"].apply(lambda x: ', '.join(x.astype(str))).reset_index()

In [3]:
JoinedTable=pd.merge(header,SpeciesGroups,on="PlotObservationID",how="left")
JoinedTable2=pd.merge(JoinedTable,LayerGroups,on="PlotObservationID",how="left")
JoinedTable3=pd.merge(JoinedTable2,CoverGroups,on="PlotObservationID",how="left")
JoinedTable4=pd.merge(JoinedTable3,CoverCodeGroups,on="PlotObservationID",how="left")

JoinedTable4.dtypes

PlotObservationID                   int64
PlotID                              int64
Bedekkingsschaal                   object
Datum van opname                   object
Lengte proefvlak (m)              float64
Breedte proefvlak (m)             float64
Opp. proefvlak (m²)               float64
Bedekking totaal (%)              float64
Bedekking boomlaag (%)            float64
Bedekking struiklaag (%)          float64
Bedekking kruidlaag (%)           float64
Bedekking moslaag (%)             float64
Hoogte (hoge) boomlaag (m)        float64
Hoogte lage boomlaag (m)          float64
Hoogte (hoge) struiklaag (m)      float64
Hoogte lage struiklaag (m)        float64
Gem. hoogte (hoge) kruidl (cm)    float64
Gem. hoogte lage kruidl. (cm)     float64
Maximale hoogte kruidlaag (cm)    float64
Associa (1)                        object
Incompleteness (1)                float64
Longitude                         float64
Latitude                          float64
Location uncertainty (m)          

- Convert into polygon

In [4]:
geometry = [Point(xy) for xy in zip(JoinedTable4['Longitude'], JoinedTable4['Latitude'])]
crs = {'init': 'epsg:4326'}
geovegdb_dataframe = GeoDataFrame(JoinedTable4[["Species name","Datum van opname"]], crs=crs, geometry=geometry)
geovegdb_dataframe.to_file(filepath+"Phragmites_australis.shp", driver='ESRI Shapefile')